In [12]:
"""Model Testing Notebook
"""
import json
import torch
from torchvision.models import vgg19
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

# Recreating trained model
model = vgg19()
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=25088, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.7, inplace=False),
    torch.nn.Linear(in_features=4096, out_features=2048, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.7, inplace=False),
    torch.nn.Linear(in_features=2048, out_features=20, bias=True)
)

# def model_stats_loader(model: torch.nn.Module) -> 


# Loading in saved model and data from EC2 training
training_dict = torch.load('trained_models/two_nodes.pt')
epochs_run1 = training_dict["EPOCHS_RUN"]
run_time1 = training_dict['RUN_TIME']
train_loss_history1 = training_dict['TRAIN_HISTORY']
valid_loss_history1 = training_dict['VALID_HISTORY']
epoch_times1 = training_dict['EPOCH_TIMES']
lowest_loss1 = training_dict['LOWEST_LOSS']

model.load_state_dict(training_dict["MODEL_STATE"])
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_data = ImageFolder(root='data/test', transform=transform)

test_loader = DataLoader(
    test_data,
    batch_size=32,
)

loss_func = F.cross_entropy

# Loading in labels map from JSON
labels_dict = {}
with open('data/labels.json') as labels_json:
    labels_dict = json.load(labels_json)

sorted_folder_list = list(sorted(labels_dict.keys()))

# Only need the first 20 out of 100 classes.
labels_map = {}
for i in range(20):
    labels_map[i] = labels_dict[sorted_folder_list[i]]

labels_map


{0: 'tench',
 1: 'goldfish',
 2: 'great white shark',
 3: 'tiger shark',
 4: 'hammerhead',
 5: 'electric ray',
 6: 'stingray',
 7: 'chicken',
 8: 'hen',
 9: 'goldfinch',
 10: 'indigo bird',
 11: 'bulbul',
 12: 'magpie',
 13: 'chickadee',
 14: 'water ouzel',
 15: 'kite',
 16: 'bald eagle',
 17: 'great grey owl',
 18: 'common newt',
 19: 'spotted salamander'}

In [13]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
classes_num = 20
class_correct = list(0. for i in range(classes_num))
class_total = list(0. for i in range(classes_num))

for data, target in test_loader:
    output = model(data)
    loss = loss_func(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    for i in range(len(target)):
        label = target[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

In [14]:
# Calculate and print average test loss
t_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(t_loss))

for i in range(classes_num):
    if class_total[i] > 0:
        percent_correct = 100 * class_correct[i] / class_total[i]
        print(
            f'Test Accuracy of Class: {labels_map[i]:19s}: {percent_correct:3.2f}%', 
            f'({np.sum(int(class_correct[i]))}/{np.sum(int(class_total[i]))})')
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' %
              (labels_map[i]))

percent_correct = 100.0 * np.sum(class_correct) / np.sum(class_total)
print(
    f'\nTest Accuracy (Overall): {percent_correct:3.2f}%',
    f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})')

Test Loss: 0.435099

Test Accuracy of Class: tench              : 92.00% (46/50)
Test Accuracy of Class: goldfish           : 98.00% (49/50)
Test Accuracy of Class: great white shark  : 74.00% (37/50)
Test Accuracy of Class: tiger shark        : 74.00% (37/50)
Test Accuracy of Class: hammerhead         : 84.00% (42/50)
Test Accuracy of Class: electric ray       : 72.00% (36/50)
Test Accuracy of Class: stingray           : 82.00% (41/50)
Test Accuracy of Class: chicken            : 68.00% (34/50)
Test Accuracy of Class: hen                : 92.00% (46/50)
Test Accuracy of Class: goldfinch          : 100.00% (50/50)
Test Accuracy of Class: indigo bird        : 98.00% (49/50)
Test Accuracy of Class: bulbul             : 92.00% (46/50)
Test Accuracy of Class: magpie             : 96.00% (48/50)
Test Accuracy of Class: chickadee          : 94.00% (47/50)
Test Accuracy of Class: water ouzel        : 88.00% (44/50)
Test Accuracy of Class: kite               : 82.00% (41/50)
Test Accuracy of C